In [30]:
import pandas as pd
import numpy as np

In [31]:
# reading movie data and user rating data
movies_df = pd.read_csv('data/movies_metadata.csv',usecols=['id','title'])

rating_df = pd.read_csv('data/ratings_small.csv',usecols=['userId', 'movieId', 'rating'],
dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [32]:
# drops Nan values
movies_df.dropna(inplace=True)

# defines Dtype of id
movies_df["id"] = movies_df['id'].astype(pd.Int64Dtype())

# renames id to movieId for later merge
movies_df = movies_df.rename(columns={'id' : 'movieId'})

movies_df

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II
...,...,...
45461,439050,Subdue
45462,111109,Century of Birthing
45463,67758,Betrayal
45464,227506,Satan Triumphant


In [33]:
rating_df.head()

,userId,movieId,rating
0,1,31,2.500
1,1,1029,3.000
2,1,1061,3.000
3,1,1129,2.000
4,1,1172,4.000


In [34]:
# merges the to dataframes
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1371,2.500,Rocky III
1,4,1371,4.000,Rocky III
2,7,1371,3.000,Rocky III
3,19,1371,4.000,Rocky III
4,21,1371,3.000,Rocky III


In [35]:
#combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

# groups all of same title, counts all the ratings and remanes ratings to totalRatingCount
movie_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()

,title,totalRatingCount
0,!Women Art Revolution,2
1,'Gator Bait,1
2,'Twas the Night Before Christmas,2
3,...And God Created Woman,1
4,00 Schneider - Jagd auf Nihil Baxter,2


In [36]:
# merges the totalRatingCount on title
rating_with_totalRatingCount = df.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1371,2.500,Rocky III,47
1,4,1371,4.000,Rocky III,47
2,7,1371,3.000,Rocky III,47
3,19,1371,4.000,Rocky III,47
4,21,1371,3.000,Rocky III,47


In [37]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   2794.000
mean      16.104
std       31.481
min        1.000
25%        1.000
50%        4.000
75%       15.750
max      324.000
Name: totalRatingCount, dtype: float64


In [38]:
# removes all movies with less then 50 ppl rating them.
popularity_threshold = 10
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.shape

(39675, 5)

In [39]:
# puts all movie and user in a metrix with ratings
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Items or Less,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2 Days in Paris,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"20,000 Leagues Under the Sea",0.000,0.000,0.000,3.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
24 Hour Party People,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000


In [40]:
movie_features_df.shape

(945, 671)

In [41]:
# from scipy.sparse import csr_matrix

# movie_features_df_matrix = csr_matrix(movie_features_df.values)

# type(movie_features_df_matrix)

In [42]:
# from sklearn.neighbors import NearestNeighbors

# model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
# model_knn.fit(movie_features_df)

from sklearn.cluster import estimate_bandwidth
bw = estimate_bandwidth(movie_features_df)

In [43]:
from sklearn.cluster import MeanShift
analyzer = MeanShift(bandwidth=bw) 
analyzer.fit(movie_features_df)

MeanShift(bandwidth=25.209509341931216)

In [54]:
labels = analyzer.labels_
print(labels)
print('\n\n',np.unique(labels))


movie_features_df['labels'] = labels

movie_features_df

[  0   0  87  38   0   0   0   0   0   0   0   0   4 131   0   0   0   0
   0 114  26   0   0   0   0  24   0 108   0   0   0   0   0   0   0   0
   0   0   0   0   0   0  76   0   0   0  79   0  43   0   0   0   0   0
   0   0   0   0   0   0   0   0   0 132   0   0   0   0   0   0 169  62
 175   0 144   0   0 125   0  35   0 171   0   0   0   0   0 164 103   0
   0   0 138   0   0   0   0  25 104   0   0 147   0   0   0 127   0   0
   0   0   0   0   0   0   0   0   0   0   0  32   0   0   0   0   0   0
   0   0   0  54 150   0   0  91   0   0   0   0   0   0   0  66   0   0
   0 142   0   0   0   0   0   0   0   0   0   0 174 156   0   0  20   0
   0   0   0 187   0   0   0   0   0   0   0   0   0   0  13   0   0 160
   0  49   0   0   0   8  56   0   0   0 177   0   0   0   0   0   0   0
   0 155   0  39   0 118   0   0   0   0   0   0   0   0   0   0   0   0
   0 153   0   0   0   0   0  77 193   0   0   0   0 172   0 116   0  52
   0   0  90   0   0   0   0   0   0   0   0   0   

userId,1,2,3,4,5,6,7,8,9,10,...,663,664,665,666,667,668,669,670,671,labels
title,,,,,,,,,,,,,,,,,,,,,
10 Items or Less,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0
2 Days in Paris,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0
"20,000 Leagues Under the Sea",0.000,0.000,0.000,3.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,87
2001: A Space Odyssey,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,38
24 Hour Party People,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zatoichi,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,141
Zodiac,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0
eXistenZ,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0


In [76]:
movie_features_df[movie_features_df['labels'] == 47].shape

(1, 672)

In [45]:
# randomly picks a movie
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
query_index = 1

870


In [46]:
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [47]:
indices

array([[  1, 527, 285,  22, 622, 251]], dtype=int64)

In [48]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Items or Less,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2 Days in Paris,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"20,000 Leagues Under the Sea",0.000,0.000,0.000,3.000,0.000,2.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
24 Hour Party People,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000


In [49]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 2 Days in Paris:

1: Peas at 5:30, with distance of 0.4842718839645386:
2: Frankenstein Created Woman, with distance of 0.5516020655632019:
3: A Friend of Mine, with distance of 0.5753188133239746:
4: Sleepy Hollow, with distance of 0.6028894186019897:
5: Erkan & Stefan 2, with distance of 0.6072729229927063:


In [50]:
indices.flatten()[i]

251